# Probabilistic Graphical Models - HWK 3 

## Paul Dufossé & Matthieu Mazzolini

Final version of our homework (January 4 2017)
link to the GitHub IPython Notebook : https://github.com/mazzma12/graphic_model/blob/master/PGM_HWK3_mazzolini_dufosse.ipynb

In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import multivariate_normal as mvn

from plotly.offline import init_notebook_mode, iplot

init_notebook_mode()

import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
data = pd.read_csv('EMGaussian.data', delim_whitespace=True, header=None, names=['x', 'y'])
test = pd.read_csv('EMGaussian.test', delim_whitespace=True, header=None, names=['x', 'y'])

### 1) We implement the alpha & beta recursions

In [ ]:
def lse(v):
    #log(sum(exp)) function 
    return np.log(np.exp(v).sum())

def compute_cond_proba(X, pi, mu, sigma):
    # Compute a T.K matrix with the probability of the observation t given the states is k
    P =np.zeros((T, K))
    for t in range(T):
        for k in range(K):
            P[t, k] = mvn.pdf(X[t], mu[k], sigma[k])
    return P 

def log_alpha_rec(X, A, P, pi, mu, sigma):
    (T, p) = X.shape
    alpha = np.ones((T,K))
    # The LOG of the messages alpha are contained in the matrix Alpha. 
    # The t-th row corresponds to the time t
    # The k-th column corresponds to the case where the state takes the value k

    # Computation of the first alpha(q_0)
    for k in range(K):
        alpha[0,k] = np.log(P[0, k]) + np.log(pi[k])

    for t in range(1,T):
        for k in range(K):
            # Alpha message formula p9 chp 12.4 of the book
            log_proba_vec = alpha[t-1] + np.log(A[:,k])
            m = max(log_proba_vec)
            alpha[t, k] = m + np.log(P[t, k]) + lse(log_proba_vec - m)
            
    return alpha

def log_beta_rec(X, A, P, pi, mu, sigma):
    (T,p) = X.shape
    beta = np.ones((T,K))
    
    # Initialization of the last time T
    # Maybe it should be something else,
    for k in range(K):
        beta[T-1,k] = np.log(P[T-1, k]) + np.log(pi[k])
    
    for t in range(T-1)[::-1]:
        for k in range(K):
            # Beta message formula 12.30 p10 chp 12.4 of the book
            # This time there is no constant term because the conditional probability
            # depends on q_(t+1) the index of the sum
            
            # Therefore we have to run another loop to compute 
            # this cond probability for K values
            cond_proba = [np.log(P[t+1, j]) for j in range(K)]
            log_proba_vec = beta[t+1] + np.log(A[k,:]) + cond_proba
            m = max(log_proba_vec)
            beta[t, k] = lse(log_proba_vec-m) + m 
            
    return beta


Then we can compute the probabilities : gamma(qt) and ksi(qt, qt1) (qt1 stands for q(t+1))
gamma and ksi are the notations used in the book. 

In [ ]:
X = np.array(data)
(T, p) = X.shape
K  =4
pi = 1.0/4 * np.ones(4)

A = np.eye(K)*(1/2-1/6) + np.ones((K,K))*1/6

#we are not sure of the parameters for the previous homework 
#so we chose to use the EM estimation from the scikit algorithm
from sklearn.mixture import GaussianMixture

model = GaussianMixture(n_components=4, covariance_type='full')
model.fit(X)
mu_ = model.means_
sigma_ = model.covariances_

In [ ]:
def compute_filtering(log_alpha, log_beta):
    filtering = np.zeros((T, K))
    for t in range(T):
        ai = log_alpha[t, :] + log_beta[t, :]
        max_ai = np.max(ai)
        log_normalization = max_ai + lse(log_alpha[t, :] + log_beta[t, :] - max_ai)
        filtering[t, :] = np.exp(log_alpha[t, :] + log_beta[t, :] - log_normalization)
        filtering[t,:] /= np.sum(filtering[t,:])
    return filtering

In [ ]:
#p_qt 

P = compute_cond_proba(X, pi, mu_, sigma_)

log_alpha = log_alpha_rec(X, A, P, pi, mu_, sigma_)
log_beta = log_beta_rec(X, A, P, pi, mu_, sigma_)
gamma = compute_filtering(log_alpha, log_beta)


In [ ]:
def plot_states_probability(data):
    # Plot the conditional probability of the first 100 states fiven the observations
    for k in range(K):
        trace=go.Scatter(
            x=np.arange(100),
            y=data[0:99,k]
        )
        layout = go.Layout(title='probability of being at state '+str(k+1)+' at time t given all the observation',
                          xaxis=dict(title='time'),
                          yaxis=dict(title='probability')
                          )
        fig = go.Figure(data=[trace], layout=layout)
        iplot(fig, filename="plot")
        
plot_states_probability(gamma)

From the gamma matrix we compute the most probable sequence : we set 1 for the most probable state and 0 for the others

In [ ]:
def most_probable (gamma):
    # the most probable state is set to one and the others to zero
    m_p = np.zeros((T,K))
    for t in range(T):
        i = np.argmax(gamma[t,:])
        m_p[t,i] = 1
    return m_p

In [ ]:
plot_states_probability(most_probable(gamma))

From alpha and gamma we compute the ksi variables and will use them in the next EM algorithm 

In [ ]:
def compute_log_ksi(qt, qt1, t, A, P, pi, mu, sigma):
    return log_alpha[t, qt]+np.log(P[t+1, qt1])+np.log(gamma[t, qt1])+np.log(A[qt, qt1]) -log_alpha[t+1, qt1]

def compute_cooccurrence(P, A, pi, mu, sigma):
    ksi = np.zeros(((T-1, K, K)))
    for t in range(T-1):
        for k in range(K):
            for l in range(K):
                ksi[t, k, l] = compute_log_ksi(k, l, t, A, P, pi, mu, sigma)
    return ksi

In [ ]:
ksi = compute_cooccurrence(P, A, pi, mu_, sigma_)

### 3) You can see the estimation equations for the EM algo in annex 

### 4) We implement now the EM algorithm. The first E-step has already been done.


In [ ]:
def compute_loglikelihood(X, A, P, pi, gamma):
    most = most_probable(gamma)
    M = np.zeros((K,K))
    for i in range(K):
        for j in range(K):
            for t in range(T-1):
                M[i,j] += most[t,i]*most[t+1,j]
    return np.dot(most[0,:],np.log(pi)) + (np.log(A)*M).sum() + (most*np.log(P)).sum()
            

In [ ]:
def EM_hmm(X, A_iter, P_iter, pi_iter, mu_iter, sigma_iter, max_iter=30, tol=0.0000000001):
    # EM implementation for our HMM model
    tours = 0
    convergence = False
    lklh = []
    while(convergence is False and tours<max_iter):

        #E-step
        P_iter = compute_cond_proba(X, pi_iter, mu_iter, sigma_iter)
        log_alpha = log_alpha_rec(X, A_iter, P_iter, pi_iter, mu_iter, sigma_iter)
        log_beta = log_beta_rec(X, A_iter, P_iter, pi_iter, mu_iter, sigma_iter)
        gamma_iter = compute_filtering(log_alpha, log_beta)
        ksi_iter = compute_cooccurrence(P_iter, A_iter, pi_iter, mu_iter, sigma_iter)
        #M-step
        for i in range(K):
            A_iter[i,:] = ksi.sum(0)[i,:]/ksi.sum(0).sum(0)
            pi_iter = gamma_iter[0]/gamma_iter[0].sum(0)
        for i in range(K):
            mu_iter[i] = np.dot(gamma[:,i], X)/gamma[:,i].sum()
        for i in range(K):
            temp = X-mu_iter[i]
            sigma_temp = 0
            for t in range(T):
                temp = np.reshape(X[t,:] - mu_iter[i], (1, 2))
                sigma_temp += gamma[t, i] * np.dot(temp.T, temp)             
            sigma_temp /= np.sum(gamma[:,i])
            sigma_iter[i] = sigma_temp
        lklh.append(compute_loglikelihood(X, A_iter, P_iter, pi_iter, gamma_iter))
        if (tours > 1):
            if (np.abs(lklh[tours] - lklh[tours-1]) < tol ):
                convergence = True
        tours+=1

    return A_iter, P_iter, pi_iter,mu_iter, sigma_iter, gamma_iter, lklh, tours


### 5) We train the EM on our HMM 

In [ ]:
obj = EM_hmm(X, A, P, pi, mu_, sigma_)
[A_final, P_final, pi_final,mu_final, sigma_final, gamma_final, lklh, tours] = obj
print("nombre d'iterations : ", tours)
print("variation de la loglikelihood : ", lklh)

### 6) We compute and plot the loglikelihood for the train data

**Comments :** We reached the maximum in 6 iterations, and we notice that the loglikelihood value doesn't improve much. This can be explained by the fact that we initiate the algorithm with the value obtained in the previous homework, which were pretty decent. 

In [ ]:
trace = go.Scatter(x=np.arange(tours), y=(lklh), name='train data')
iplot([trace])

In [ ]:
# Computing the likelihood for the test data
X_test = np.array(test)
P_test = compute_cond_proba(X_test, pi_final, mu_final, sigma_final)
log_alpha_test = log_alpha_rec(X_test, A_final, P_test, pi_final, mu_final, sigma_final)
log_beta_test = log_beta_rec(X_test, A_final, P_test, pi_final, mu_final, sigma_final)
gamma_test = compute_filtering(log_alpha_test, log_beta_test)
likelihood_test = compute_loglikelihood(X_test, A_final, P_test, pi_final, gamma_test)
print("likelihood for test data : ", likelihood_test, "likelihood for train data : ", lklh[-1])

**Comment** We observe that the loglikelihood is a bit lower on the test data than on the train data, 
which is as usual because we trained the model on the train data

### 7) You can see the description of Viterbi algorithm in annex

### 8) Now we implement a new method : the Viterbi decoding algorithm (or max-product algorithm)

In [ ]:
def viterbi(X, A, pi, mu, sigma):
    T1=np.zeros((T, K))
    T2=np.zeros((T, K))
    
    for k in range(K):
        T1[1, k] = np.log(pi[k]) + mvn.logpdf(X[0], mu[k], sigma[k]) 
        
    for t in range(1,T):
        for k in range(K):
            T1[t,k] = np.max(T1[t-1,:] + np.log(A[k,:])) + mvn.logpdf(X[t], mu[k], sigma[k])
            T2[t,k] = np.argmax(T1[t-1,:] + np.log(A[k,:]))
            
    seq = np.zeros(T)
    seq[T-1] = np.argmax(T1[T-1,:]) 
    for t in range(T-1)[::-1]:
        seq[t-1] = T2[t, seq[t]]
    
    # We add 1 to have the states from 1 to 4 instead of 0 to 3   
    seq +=1        
    return seq

viterbi_results = viterbi(X, A, pi, mu_, sigma_)

We plot the result for the viterbi algorithm

In [ ]:
data = [(viterbi_results==k).astype(int) for k in range(1,5)]
data = np.array(data).T
print(data.shape)
plot_states_probability(data)


**Conclusion ** : We find the same result with the Viterbi algorithm than the ones with the filtering fonction used in question 2)

### 9) & 10) We used the gamma_test variable that we computed with the test data to compute the most likely states for test data

In [ ]:
plot_states_probability(gamma_test)

We run Viterbi algorithm with the result obtained from the test data : 

In [ ]:
viterbi_test_results = viterbi(X_test, A_final, pi_final, mu_final, sigma_final)
data = [(viterbi_test_results==k).astype(int) for k in range(1,5)]
data = np.array(data).T
print(data.shape)
plot_states_probability(data)

We compare the results obtained with Viterbi and the filtering

In [ ]:
# Difference between the two predictions
absolute_error = np.abs((most_probable(gamma_test)-data)).sum()
print(" #differences between the 2 algos : ", absolute_error)
print("percentage of error : ", 6/500)

We had 6 errors between the Viterbi and the filtering on the test data **which is 1.2% (decent)**